# Pattern Recognition - Apple Stock 2023-2025

Testing pattern recognition indicators: Pivots, Hammer, RSI Divergence.

In [1]:
import sys
from pathlib import Path

import plotly.graph_objects as go
import yfinance as yf
from plotly.subplots import make_subplots

sys.path.insert(0, str(Path("../..").resolve()))

from indicators.momentum import calculate_rsi
from indicators.pattern import calculate_hammer, detect_rsi_divergence, find_pivots

In [2]:
# Fetch Apple data
df = yf.download("AAPL", start="2023-01-01", end="2025-10-01", auto_adjust=True, progress=False)

# Handle MultiIndex columns from yfinance
if df.columns.nlevels == 2:
    df.columns = df.columns.get_level_values(0)

df.columns = df.columns.str.lower()
df = df.reset_index()
df.columns = df.columns.str.lower()

print(f"Data shape: {df.shape}")
df.head()

Data shape: (688, 6)


Price,date,close,high,low,open,volume
0,2023-01-03,123.211205,128.954553,122.324579,128.343772,112117500
1,2023-01-04,124.482033,126.747853,123.221057,125.004155,89113600
2,2023-01-05,123.161949,125.871079,122.905819,125.240591,80962700
3,2023-01-06,127.693573,128.353614,123.033875,124.137232,87754700
4,2023-01-09,128.215683,131.427242,127.959553,128.530934,70790800


## 1. Pivot Points

Identifies local highs and lows using lookback/lookahead windows (lb=8, rb=13). CRITICAL for ML labeling - marks turning points.

In [3]:
pivot_high, pivot_low = find_pivots(df, lb=8, rb=13, use_close=True)
df["PivotHigh"] = pivot_high
df["PivotLow"] = pivot_low

fig = go.Figure()

# Price line
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 1}))

# Pivot highs
pivot_highs = df[df["PivotHigh"] == 1]
fig.add_trace(
    go.Scatter(
        x=pivot_highs["date"],
        y=pivot_highs["close"],
        mode="markers",
        name="Pivot High",
        marker={"color": "red", "size": 10, "symbol": "triangle-down"},
    )
)

# Pivot lows
pivot_lows = df[df["PivotLow"] == 1]
fig.add_trace(
    go.Scatter(
        x=pivot_lows["date"],
        y=pivot_lows["close"],
        mode="markers",
        name="Pivot Low",
        marker={"color": "green", "size": 10, "symbol": "triangle-up"},
    )
)

fig.update_layout(height=600, title_text="Pivot Points Detection (lb=8, rb=13)")
fig.show()

print(f"Pivot Highs found: {(df['PivotHigh'] == 1).sum()}")
print(f"Pivot Lows found: {(df['PivotLow'] == 1).sum()}")

Pivot Highs found: 17
Pivot Lows found: 25


## 1.1 Pivot Points with Window Variations

Extended pivot detection allowing ±1 and ±2 day tolerance around base pivots. Adjacent days marked as pivots only if close price within 1% of base pivot (price_tolerance default).

In [11]:
# Test different window variations
pivot_high_var1, pivot_low_var1 = find_pivots(df, lb=8, rb=13, use_close=True, window_variations=[-1, 1], price_tolerance=0.01)
pivot_high_var2, pivot_low_var2 = find_pivots(df, lb=8, rb=13, use_close=True, window_variations=[-2, -1, 1, 2], price_tolerance=0.01)

# Create comparison figure
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Base Pivots (no variation)", "Pivots with ±1 day variation", "Pivots with ±2 day variation"),
)

# Row 1: Base pivots
fig.add_trace(
    go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 1}, showlegend=False),
    row=1,
    col=1,
)
pivot_highs_base = df[df["PivotHigh"] == 1]
pivot_lows_base = df[df["PivotLow"] == 1]
fig.add_trace(
    go.Scatter(
        x=pivot_highs_base["date"],
        y=pivot_highs_base["close"],
        mode="markers",
        name="Pivot High",
        marker={"color": "red", "size": 8, "symbol": "triangle-down"},
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=pivot_lows_base["date"],
        y=pivot_lows_base["close"],
        mode="markers",
        name="Pivot Low",
        marker={"color": "green", "size": 8, "symbol": "triangle-up"},
    ),
    row=1,
    col=1,
)

# Row 2: ±1 variation
fig.add_trace(
    go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 1}, showlegend=False),
    row=2,
    col=1,
)
pivot_highs_var1 = df[pivot_high_var1 == 1]
pivot_lows_var1 = df[pivot_low_var1 == 1]
fig.add_trace(
    go.Scatter(
        x=pivot_highs_var1["date"],
        y=pivot_highs_var1["close"],
        mode="markers",
        name="Pivot High ±1",
        marker={"color": "red", "size": 8, "symbol": "triangle-down"},
        showlegend=False,
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=pivot_lows_var1["date"],
        y=pivot_lows_var1["close"],
        mode="markers",
        name="Pivot Low ±1",
        marker={"color": "green", "size": 8, "symbol": "triangle-up"},
        showlegend=False,
    ),
    row=2,
    col=1,
)

# Row 3: ±2 variation
fig.add_trace(
    go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 1}, showlegend=False),
    row=3,
    col=1,
)
pivot_highs_var2 = df[pivot_high_var2 == 1]
pivot_lows_var2 = df[pivot_low_var2 == 1]
fig.add_trace(
    go.Scatter(
        x=pivot_highs_var2["date"],
        y=pivot_highs_var2["close"],
        mode="markers",
        name="Pivot High ±2",
        marker={"color": "red", "size": 8, "symbol": "triangle-down"},
        showlegend=False,
    ),
    row=3,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=pivot_lows_var2["date"],
        y=pivot_lows_var2["close"],
        mode="markers",
        name="Pivot Low ±2",
        marker={"color": "green", "size": 8, "symbol": "triangle-up"},
        showlegend=False,
    ),
    row=3,
    col=1,
)

fig.update_layout(height=1600, title_text="Pivot Detection: Window Variations Comparison (5% price tolerance)")
fig.show()

print(f"Base Pivots - Highs: {(df['PivotHigh'] == 1).sum()}, Lows: {(df['PivotLow'] == 1).sum()}")
print(f"±1 Variation - Highs: {(pivot_high_var1 == 1).sum()}, Lows: {(pivot_low_var1 == 1).sum()}")
print(f"±2 Variation - Highs: {(pivot_high_var2 == 1).sum()}, Lows: {(pivot_low_var2 == 1).sum()}")
print("\nWindow variations expand pivot labels for ML training,")
print("but only if adjacent day's close is within 5% of base pivot.")

Base Pivots - Highs: 17, Lows: 25
±1 Variation - Highs: 36, Lows: 49
±2 Variation - Highs: 46, Lows: 59

Window variations expand pivot labels for ML training,
but only if adjacent day's close is within 5% of base pivot.


## 2. Hammer Candlestick Pattern

Bullish reversal pattern: small body at top, long lower shadow (>2x body), little/no upper shadow. Signals buying pressure overcame selling.

In [5]:
df["hammer"] = calculate_hammer(df)

fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Price with Hammer Patterns", "Hammer Signal"),
)

# Price
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black"}), row=1, col=1)

# Mark hammer patterns on price
hammers = df[df["hammer"] == 100]
fig.add_trace(
    go.Scatter(
        x=hammers["date"],
        y=hammers["close"],
        mode="markers",
        name="Hammer",
        marker={"color": "orange", "size": 10, "symbol": "star"},
    ),
    row=1,
    col=1,
)

# Hammer signal
fig.add_trace(
    go.Scatter(x=df["date"], y=df["hammer"], name="Hammer", fill="tozeroy", line={"color": "orange"}), row=2, col=1
)

fig.update_layout(height=600, title_text="Hammer Candlestick Pattern (Bullish Reversal)")
fig.show()

print(f"Hammer patterns found: {(df['hammer'] == 100).sum()}")

Hammer patterns found: 17


## 3. RSI Divergence Detection

Backward-looking divergence detection - NO LOOKAHEAD BIAS.

- Bullish divergence: Price makes lower low, RSI makes higher low
- Bearish divergence: Price makes higher high, RSI makes lower high

Uses rolling window to detect local extrema, then compares consecutive extrema. Safe for real-time trading and ML features.

In [6]:
# Calculate RSI
df["RSI"] = calculate_rsi(df, period=14)

# Detect divergence
df = detect_rsi_divergence(df, rsi_col="RSI", price_col="close", lookback_window=5, max_lookback=55, min_distance=5)

# Visualize
fig = make_subplots(
    rows=4,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Price with Divergences", "RSI", "Bullish Divergence", "Bearish Divergence"),
)

# Row 1: Price with divergence markers
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black"}), row=1, col=1)

bullish = df[df["Bullish_Divergence"] == 1]
fig.add_trace(
    go.Scatter(
        x=bullish["date"],
        y=bullish["close"],
        mode="markers",
        name="Bullish Div",
        marker={"color": "green", "size": 12, "symbol": "triangle-up"},
    ),
    row=1,
    col=1,
)

bearish = df[df["Bearish_Divergence"] == 1]
fig.add_trace(
    go.Scatter(
        x=bearish["date"],
        y=bearish["close"],
        mode="markers",
        name="Bearish Div",
        marker={"color": "red", "size": 12, "symbol": "triangle-down"},
    ),
    row=1,
    col=1,
)

# Row 2: RSI
fig.add_trace(go.Scatter(x=df["date"], y=df["RSI"], name="RSI", line={"color": "blue"}), row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)

# Row 3: Bullish signal
fig.add_trace(
    go.Scatter(x=df["date"], y=df["Bullish_Divergence"], name="Bullish", fill="tozeroy", line={"color": "green"}),
    row=3,
    col=1,
)

# Row 4: Bearish signal
fig.add_trace(
    go.Scatter(x=df["date"], y=df["Bearish_Divergence"], name="Bearish", fill="tozeroy", line={"color": "red"}),
    row=4,
    col=1,
)

fig.update_layout(height=1000, title_text="RSI Divergence (Backward-Looking)")
fig.show()

print(f"Bullish divergences: {(df['Bullish_Divergence'] == 1).sum()}")
print(f"Bearish divergences: {(df['Bearish_Divergence'] == 1).sum()}")
print(f"Avg strength: {df[df['Divergence_Strength'] > 0]['Divergence_Strength'].mean():.4f}")

Bullish divergences: 14
Bearish divergences: 40
Avg strength: 0.0233


## Combined View - All Pattern Indicators

In [7]:
fig = go.Figure()

# Price
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 1.5}))

# Pivot highs (red triangles) - forward-looking, for labels
pivot_highs = df[df["PivotHigh"] == 1]
fig.add_trace(
    go.Scatter(
        x=pivot_highs["date"],
        y=pivot_highs["close"],
        mode="markers",
        name="Pivot High",
        marker={"color": "red", "size": 10, "symbol": "triangle-down"},
    )
)

# Pivot lows (green triangles) - forward-looking, for labels
pivot_lows = df[df["PivotLow"] == 1]
fig.add_trace(
    go.Scatter(
        x=pivot_lows["date"],
        y=pivot_lows["close"],
        mode="markers",
        name="Pivot Low",
        marker={"color": "green", "size": 10, "symbol": "triangle-up"},
    )
)

# Hammer patterns (orange stars) - backward-looking, safe for features
hammers = df[df["hammer"] == 100]
fig.add_trace(
    go.Scatter(
        x=hammers["date"],
        y=hammers["close"],
        mode="markers",
        name="Hammer",
        marker={"color": "orange", "size": 12, "symbol": "star"},
    )
)

# Bullish RSI divergences (large green circles) - backward-looking, safe for features
bullish_divs = df[df["Bullish_Divergence"] == 1]
fig.add_trace(
    go.Scatter(
        x=bullish_divs["date"],
        y=bullish_divs["close"],
        mode="markers",
        name="Bullish RSI Div",
        marker={"color": "lime", "size": 15, "symbol": "circle", "line": {"color": "darkgreen", "width": 2}},
    )
)

# Bearish RSI divergences (large red circles) - backward-looking, safe for features
bearish_divs = df[df["Bearish_Divergence"] == 1]
fig.add_trace(
    go.Scatter(
        x=bearish_divs["date"],
        y=bearish_divs["close"],
        mode="markers",
        name="Bearish RSI Div",
        marker={"color": "pink", "size": 15, "symbol": "circle", "line": {"color": "darkred", "width": 2}},
    )
)

fig.update_layout(
    height=700, title_text="Pattern Recognition - Complete Overview", xaxis_title="Date", yaxis_title="Price"
)
fig.show()

print("\n=== Pattern Summary ===")
print(f"Pivot Highs: {(df['PivotHigh'] == 1).sum()} (forward-looking, ML labels only)")
print(f"Pivot Lows: {(df['PivotLow'] == 1).sum()} (forward-looking, ML labels only)")
print(f"Hammer: {(df['hammer'] == 100).sum()} (backward-looking, safe for ML features)")
print(f"Bullish RSI Div: {(df['Bullish_Divergence'] == 1).sum()} (backward-looking, safe for ML features)")
print(f"Bearish RSI Div: {(df['Bearish_Divergence'] == 1).sum()} (backward-looking, safe for ML features)")


=== Pattern Summary ===
Pivot Highs: 17 (forward-looking, ML labels only)
Pivot Lows: 25 (forward-looking, ML labels only)
Hammer: 17 (backward-looking, safe for ML features)
Bullish RSI Div: 14 (backward-looking, safe for ML features)
Bearish RSI Div: 40 (backward-looking, safe for ML features)
